<a href="https://colab.research.google.com/github/AlbertV100/Spring/blob/main/Thoracic/Thoracic_Optimised_DenseNet121_Fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thoracic NIH (Subset) — Optimised DenseNet121

Fresh Colab notebook patched to avoid:

- **All-NaN per-label AUROC / Macro AUROC**
- label NaNs / dtype drift
- bad splits that drop positives in VAL/TEST
- unstable mixed-precision metrics

Main upgrades:
- multi-label stratified split (**patient_id grouped if available**)
- tf.data w/ cache + prefetch + AUTOTUNE
- mixed precision on A100 (float32 outputs)
- AUROC evaluation that **skips impossible labels** and prints per-label counts


In [ ]:
# Install dependency for multilabel stratified split
!pip -q install iterative-stratification


In [ ]:
# Runtime config (A100)
import os, random, numpy as np, pandas as pd
import tensorflow as tf

SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("TensorFlow:", tf.__version__)
print("GPU devices:", tf.config.list_physical_devices("GPU"))

# Mixed precision is a big win on A100
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")
print("Mixed precision policy:", mixed_precision.global_policy())

# Optional: XLA (usually helpful on A100; disable if you see issues)
try:
    tf.config.optimizer.set_jit(True)
    print("XLA JIT enabled")
except Exception as e:
    print("XLA JIT not enabled:", e)

AUTOTUNE = tf.data.AUTOTUNE


TensorFlow: 2.19.0
GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Mixed precision policy: <DTypePolicy "mixed_float16">
XLA JIT enabled


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


## Paths (edit if your Drive folder differs)


In [ ]:
BASE_DIR = "/content/drive/MyDrive/Thoracic_subset"
CSV_PATH = f"{BASE_DIR}/cleaned_metadata_subset.csv"
IMG_DIR  = f"{BASE_DIR}/images"


## Load metadata + targets


In [ ]:
TARGETS = ["Atelectasis","Cardiomegaly","Effusion","Infiltration","Mass","Nodule","Pneumonia","Pneumothorax"]
TARGET_COLS = [f"label_{t}" for t in TARGETS]

df = pd.read_csv(CSV_PATH)
df.columns = [c.strip() for c in df.columns]

assert "image_index" in df.columns, "CSV must contain image_index"
missing = [c for c in TARGET_COLS if c not in df.columns]
assert not missing, f"Missing target columns: {missing}"

# Fix label NaNs/dtypes (prevents NaN AUROC due to NaN y_true)
df[TARGET_COLS] = (
    df[TARGET_COLS]
      .apply(pd.to_numeric, errors="coerce")
      .fillna(0)
      .clip(0, 1)
      .astype(np.float32)
)

# Build image paths + keep only existing files
df["image_path"] = df["image_index"].astype(str).apply(lambda x: os.path.join(IMG_DIR, x))
exists_mask = df["image_path"].apply(tf.io.gfile.exists)
df = df.loc[exists_mask].reset_index(drop=True)

print("Rows:", len(df))
print("Positives per label:")
print(df[TARGET_COLS].sum().astype(int))


Rows: 12000
Positives per label:
label_Atelectasis     1016
label_Cardiomegaly     583
label_Effusion        1242
label_Infiltration    1414
label_Mass             770
label_Nodule           758
label_Pneumonia        476
label_Pneumothorax     709
dtype: int64


## Multi-label stratified split (patient-level if available)


In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

def multilabel_split_df(df, label_cols, group_col=None, seed=42, test_size=0.10, val_size=0.10):
    assert 0 < test_size < 1 and 0 < val_size < 1 and (test_size + val_size) < 1

    if group_col and group_col in df.columns:
        g = df.groupby(group_col)[label_cols].max()
        groups = g.index.values
        Yg = g.values.astype(np.float32)

        msss1 = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=(test_size + val_size), random_state=seed)
        train_g_idx, tmp_g_idx = next(msss1.split(groups, Yg))

        train_groups = set(groups[train_g_idx])
        tmp_groups   = set(groups[tmp_g_idx])

        train_df = df[df[group_col].isin(train_groups)].copy()
        tmp_df   = df[df[group_col].isin(tmp_groups)].copy()

        g2 = tmp_df.groupby(group_col)[label_cols].max()
        groups2 = g2.index.values
        Yg2 = g2.values.astype(np.float32)

        test_frac_of_tmp = test_size / (test_size + val_size)
        msss2 = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=test_frac_of_tmp, random_state=seed)
        val_g_idx, test_g_idx = next(msss2.split(groups2, Yg2))

        val_groups  = set(groups2[val_g_idx])
        test_groups = set(groups2[test_g_idx])

        val_df  = tmp_df[tmp_df[group_col].isin(val_groups)].copy()
        test_df = tmp_df[tmp_df[group_col].isin(test_groups)].copy()

        mode = f"grouped by {group_col}"
    else:
        X = df.index.values
        Y = df[label_cols].values.astype(np.float32)

        msss1 = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=(test_size + val_size), random_state=seed)
        train_idx, tmp_idx = next(msss1.split(X, Y))
        train_df = df.iloc[train_idx].copy()
        tmp_df   = df.iloc[tmp_idx].copy()

        X2 = tmp_df.index.values
        Y2 = tmp_df[label_cols].values.astype(np.float32)

        test_frac_of_tmp = test_size / (test_size + val_size)
        msss2 = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=test_frac_of_tmp, random_state=seed)
        val_idx, test_idx = next(msss2.split(X2, Y2))

        val_df  = tmp_df.iloc[val_idx].copy()
        test_df = tmp_df.iloc[test_idx].copy()

        mode = "sample-level"

    train_df = train_df.reset_index(drop=True)
    val_df   = val_df.reset_index(drop=True)
    test_df  = test_df.reset_index(drop=True)

    # No leakage
    assert set(train_df["image_path"]).isdisjoint(set(val_df["image_path"]))
    assert set(train_df["image_path"]).isdisjoint(set(test_df["image_path"]))
    assert set(val_df["image_path"]).isdisjoint(set(test_df["image_path"]))

    return train_df, val_df, test_df, mode

def split_report(name, d, label_cols):
    pos = d[label_cols].sum().astype(int)
    neg = (len(d) - pos).astype(int)
    print(f"\n{name}: n={len(d)}")
    print("positives:", pos.to_dict())
    print("labels with >=1 pos:", int((pos > 0).sum()), "/", len(label_cols))
    print("labels with both classes:", int(((pos > 0) & (neg > 0)).sum()), "/", len(label_cols))

GROUP_COL = "patient_id"  # if present, prevents leakage
train_df, val_df, test_df, mode = multilabel_split_df(df, TARGET_COLS, group_col=GROUP_COL, seed=SEED, test_size=0.10, val_size=0.10)
print("Split mode:", mode)

split_report("TRAIN", train_df, TARGET_COLS)
split_report("VAL",   val_df,   TARGET_COLS)
split_report("TEST",  test_df,  TARGET_COLS)


Split mode: grouped by patient_id

TRAIN: n=9622
positives: {'label_Atelectasis': 810, 'label_Cardiomegaly': 464, 'label_Effusion': 983, 'label_Infiltration': 1135, 'label_Mass': 611, 'label_Nodule': 606, 'label_Pneumonia': 385, 'label_Pneumothorax': 561}
labels with >=1 pos: 8 / 8
labels with both classes: 8 / 8

VAL: n=1184
positives: {'label_Atelectasis': 104, 'label_Cardiomegaly': 58, 'label_Effusion': 124, 'label_Infiltration': 138, 'label_Mass': 77, 'label_Nodule': 77, 'label_Pneumonia': 45, 'label_Pneumothorax': 80}
labels with >=1 pos: 8 / 8
labels with both classes: 8 / 8

TEST: n=1194
positives: {'label_Atelectasis': 102, 'label_Cardiomegaly': 61, 'label_Effusion': 135, 'label_Infiltration': 141, 'label_Mass': 82, 'label_Nodule': 75, 'label_Pneumonia': 46, 'label_Pneumothorax': 68}
labels with >=1 pos: 8 / 8
labels with both classes: 8 / 8


## tf.data pipeline (A100-friendly)


In [ ]:
from tensorflow.keras.applications.densenet import preprocess_input

IMG_SIZE = 224
BATCH = 64
NUM_LABELS = len(TARGET_COLS)

def decode_and_resize(path, y, img_size):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, [img_size, img_size], method="bilinear")
    img = tf.cast(img, tf.float32)
    img = preprocess_input(img)
    return img, y

def make_ds(dframe, training, img_size=224, batch=64):
    paths = dframe["image_path"].values
    labels = dframe[TARGET_COLS].values.astype(np.float32)

    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if training:
        ds = ds.shuffle(buffer_size=min(len(dframe), 5000), seed=SEED, reshuffle_each_iteration=True)
    ds = ds.map(lambda p,y: decode_and_resize(p, y, img_size), num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch, drop_remainder=False)
    ds = ds.prefetch(AUTOTUNE)
    return ds

train_ds = make_ds(train_df, training=True,  img_size=IMG_SIZE, batch=BATCH)
val_ds   = make_ds(val_df,   training=False, img_size=IMG_SIZE, batch=BATCH)
test_ds  = make_ds(test_df,  training=False, img_size=IMG_SIZE, batch=BATCH)

x0, y0 = next(iter(train_ds))
print("Batch images:", x0.shape, x0.dtype)
print("Batch labels:", y0.shape, y0.dtype)


Batch images: (64, 224, 224, 3) <dtype: 'float32'>
Batch labels: (64, 8) <dtype: 'float32'>


## Model + training


In [ ]:
def build_model(img_size, n_labels):
    inputs = tf.keras.Input(shape=(img_size, img_size, 3))
    base = tf.keras.applications.DenseNet121(
        include_top=False,
        weights="imagenet",
        input_shape=(img_size, img_size, 3),
    )
    x = base(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    # Output float32 for stable loss/metrics in mixed precision
    outputs = tf.keras.layers.Dense(n_labels, activation="sigmoid", dtype="float32")(x)
    model = tf.keras.Model(inputs, outputs)
    return model, base

model, base = build_model(IMG_SIZE, NUM_LABELS)
base.trainable = False

LR_FROZEN = 1e-3
opt1 = tf.keras.optimizers.AdamW(learning_rate=LR_FROZEN, weight_decay=1e-4)

model.compile(
    optimizer=opt1,
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.AUC(curve="ROC", multi_label=True, num_labels=NUM_LABELS, name="auc")]
)

model.summary()


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ densenet121 (Functional)        │ (None, 7, 7, 1024)     │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │         8,200 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,045,704 (26.88 MB)

 Trainable params: 8,200 (32.03 KB)

 Non-trainable params: 7,037,504 (26.85 MB)

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

CKPT_DIR = "/content/checkpoints"
os.makedirs(CKPT_DIR, exist_ok=True)
ckpt_path = os.path.join(CKPT_DIR, f"densenet121_{IMG_SIZE}px_best.keras")

callbacks = [
    ModelCheckpoint(ckpt_path, monitor="val_auc", mode="max", save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor="val_auc", mode="max", factor=0.5, patience=2, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor="val_auc", mode="max", patience=4, restore_best_weights=True, verbose=1),
]

EPOCHS_FROZEN = 1
history1 = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_FROZEN, callbacks=callbacks)


151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - auc: 0.5036 - loss: 0.3084
Epoch 1: val_auc improved from -inf to 0.63786, saving model to /content/checkpoints/densenet121_224px_best.keras
151/151 ━━━━━━━━━━━━━━━━━━━━ 719s 5s/step - auc: 0.5038 - loss: 0.3082 - val_auc: 0.6379 - val_loss: 0.2541 - learning_rate: 0.0010
Restoring model weights from the end of the best epoch: 1.


## Fine-tune (unfreeze last N layers)


In [ ]:
N_UNFREEZE = 200

base.trainable = True
for layer in base.layers[:-N_UNFREEZE]:
    layer.trainable = False

LR_FT = 3e-5
opt2 = tf.keras.optimizers.AdamW(learning_rate=LR_FT, weight_decay=1e-4)

model.compile(
    optimizer=opt2,
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.AUC(curve="ROC", multi_label=True, num_labels=NUM_LABELS, name="auc")]
)

EPOCHS_FT = 15
history2 = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_FT, callbacks=callbacks)


Epoch 1/15
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - auc: 0.6041 - loss: 0.2671   
Epoch 1: val_auc improved from 0.63786 to 0.72243, saving model to /content/checkpoints/densenet121_224px_best.keras
151/151 ━━━━━━━━━━━━━━━━━━━━ 357s 1s/step - auc: 0.6043 - loss: 0.2670 - val_auc: 0.7224 - val_loss: 0.2370 - learning_rate: 3.0000e-05
Epoch 2/15
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step - auc: 0.7492 - loss: 0.2278
Epoch 2: val_auc improved from 0.72243 to 0.73939, saving model to /content/checkpoints/densenet121_224px_best.keras
151/151 ━━━━━━━━━━━━━━━━━━━━ 34s 221ms/step - auc: 0.7491 - loss: 0.2278 - val_auc: 0.7394 - val_loss: 0.2336 - learning_rate: 3.0000e-05
Epoch 3/15
151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - auc: 0.8107 - loss: 0.2051
Epoch 3: val_auc improved from 0.73939 to 0.75089, saving model to /content/checkpoints/densenet121_224px_best.keras
151/151 ━━━━━━━━━━━━━━━━━━━━ 33s 220ms/step - auc: 0.8106 - loss: 0.2051 - val_auc: 0.7509 - val_loss: 0.2333 - learning_rate:

## Evaluation (robust AUROC; no all-NaN macro)


In [ ]:
from sklearn.metrics import roc_auc_score
import json, pathlib

# Load best checkpoint if present
if tf.io.gfile.exists(ckpt_path):
    model = tf.keras.models.load_model(ckpt_path)
    print("Loaded best checkpoint:", ckpt_path)

pred = model.predict(test_ds, verbose=1)
Y_test = test_df[TARGET_COLS].values.astype(np.float32)

print("pred shape:", pred.shape, "Y_test shape:", Y_test.shape)
print("NaNs in pred:", int(np.isnan(pred).sum()), "NaNs in Y_test:", int(np.isnan(Y_test).sum()))

pos = Y_test.sum(axis=0)
neg = (Y_test == 0).sum(axis=0)

print("\nTEST counts:")
for i, name in enumerate(TARGETS):
    print(f"  {name:14s} pos={int(pos[i]):4d} neg={int(neg[i]):4d}")

per_label = []
valid_aucs = []
skipped = []

for i, name in enumerate(TARGETS):
    yt = Y_test[:, i]
    yp = pred[:, i]
    if yt.min() == yt.max():
        per_label.append((name, float("nan")))
        skipped.append(name)
    else:
        auc_i = roc_auc_score(yt, yp)
        per_label.append((name, float(auc_i)))
        valid_aucs.append(float(auc_i))

macro_auc = float(np.mean(valid_aucs)) if valid_aucs else float("nan")

print("\nPer-label AUROC:")
for name, auc_i in per_label:
    if np.isnan(auc_i):
        print(f"  {name:14s} nan (only one class in TEST)")
    else:
        print(f"  {name:14s} {auc_i:.4f}")

print("\nMacro AUROC (scored labels only):", macro_auc)
print("Scored labels:", len(valid_aucs), "/", len(TARGETS))
if skipped:
    print("Skipped labels:", skipped)

# Save metrics to Drive-friendly location
pathlib.Path("/content/outputs").mkdir(exist_ok=True)
metrics_out = {
    "macro_auc_scored_only": macro_auc,
    "per_label_auc": {k: v for k, v in per_label},
    "test_pos_counts": {TARGETS[i]: int(pos[i]) for i in range(len(TARGETS))},
    "test_n": int(len(test_df)),
    "img_size": IMG_SIZE,
    "batch": BATCH,
    "n_unfreeze": N_UNFREEZE,
    "lr_frozen": LR_FROZEN,
    "lr_finetune": LR_FT,
}
with open("/content/outputs/metrics.json", "w") as f:
    json.dump(metrics_out, f, indent=2)
print("Saved: /content/outputs/metrics.json")


Loaded best checkpoint: /content/checkpoints/densenet121_224px_best.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 98s 4s/step
pred shape: (1194, 8) Y_test shape: (1194, 8)
NaNs in pred: 0 NaNs in Y_test: 0

TEST counts:
  Atelectasis    pos= 102 neg=1092
  Cardiomegaly   pos=  61 neg=1133
  Effusion       pos= 135 neg=1059
  Infiltration   pos= 141 neg=1053
  Mass           pos=  82 neg=1112
  Nodule         pos=  75 neg=1119
  Pneumonia      pos=  46 neg=1148
  Pneumothorax   pos=  68 neg=1126

Per-label AUROC:
  Atelectasis    0.7463
  Cardiomegaly   0.8046
  Effusion       0.8029
  Infiltration   0.7094
  Mass           0.6833
  Nodule         0.6264
  Pneumonia      0.6040
  Pneumothorax   0.7608

Macro AUROC (scored labels only): 0.7172313828670229
Scored labels: 8 / 8
Saved: /content/outputs/metrics.json


## Optional: High-res pass (320px)


In [ ]:
HIGH_RES = True

if HIGH_RES:
    IMG_SIZE = 320
    BATCH = 32

    train_ds = make_ds(train_df, training=True,  img_size=IMG_SIZE, batch=BATCH)
    val_ds   = make_ds(val_df,   training=False, img_size=IMG_SIZE, batch=BATCH)
    test_ds  = make_ds(test_df,  training=False, img_size=IMG_SIZE, batch=BATCH)

    model, base = build_model(IMG_SIZE, NUM_LABELS)
    base.trainable = False

    ckpt_path = os.path.join(CKPT_DIR, f"densenet121_{IMG_SIZE}px_best.keras")
    callbacks[0] = ModelCheckpoint(ckpt_path, monitor="val_auc", mode="max", save_best_only=True, verbose=1)

    model.compile(
        optimizer=tf.keras.optimizers.AdamW(learning_rate=LR_FROZEN, weight_decay=1e-4),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.AUC(curve="ROC", multi_label=True, num_labels=NUM_LABELS, name="auc")]
    )
    model.fit(train_ds, validation_data=val_ds, epochs=1, callbacks=callbacks)

    # fine-tune
    N_UNFREEZE = 200
    base.trainable = True
    for layer in base.layers[:-N_UNFREEZE]:
        layer.trainable = False

    model.compile(
        optimizer=tf.keras.optimizers.AdamW(learning_rate=LR_FT, weight_decay=1e-4),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.AUC(curve="ROC", multi_label=True, num_labels=NUM_LABELS, name="auc")]
    )
    model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=callbacks)

    print("High-res run complete.")


301/301 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - auc: 0.5117 - loss: 0.3146
Epoch 1: val_auc improved from -inf to 0.65786, saving model to /content/checkpoints/densenet121_320px_best.keras
301/301 ━━━━━━━━━━━━━━━━━━━━ 107s 233ms/step - auc: 0.5118 - loss: 0.3145 - val_auc: 0.6579 - val_loss: 0.2538 - learning_rate: 0.0010
Restoring model weights from the end of the best epoch: 1.
Epoch 1/10
301/301 ━━━━━━━━━━━━━━━━━━━━ 0s 622ms/step - auc: 0.6205 - loss: 0.2591
Epoch 1: val_auc improved from 0.65786 to 0.73517, saving model to /content/checkpoints/densenet121_320px_best.keras
301/301 ━━━━━━━━━━━━━━━━━━━━ 369s 688ms/step - auc: 0.6206 - loss: 0.2591 - val_auc: 0.7352 - val_loss: 0.2340 - learning_rate: 3.0000e-05
Epoch 2/10
301/301 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - auc: 0.7712 - loss: 0.2198
Epoch 2: val_auc improved from 0.73517 to 0.75511, saving model to /content/checkpoints/densenet121_320px_best.keras
301/301 ━━━━━━━━━━━━━━━━━━━━ 36s 119ms/step - auc: 0.7711 - loss: 0.2198 - val_au

In [ ]:
# 1) Load best checkpoint saved by ModelCheckpoint
model.load_weights(ckpt_path)

# 2) Evaluate on val + test
val_metrics = model.evaluate(val_ds, verbose=1)
test_metrics = model.evaluate(test_ds, verbose=1)

print("Val metrics:", dict(zip(model.metrics_names, val_metrics)))
print("Test metrics:", dict(zip(model.metrics_names, test_metrics)))


37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 95ms/step - auc: 0.7575 - loss: 0.2285
38/38 ━━━━━━━━━━━━━━━━━━━━ 20s 534ms/step - auc: 0.7408 - loss: 0.2303
Val metrics: {'loss': 0.23108196258544922, 'compile_metrics': 0.7665917277336121}
Test metrics: {'loss': 0.23693576455116272, 'compile_metrics': 0.7399544715881348}


In [ ]:
auc_metric = tf.keras.metrics.AUC(curve="ROC", multi_label=True, num_labels=NUM_LABELS, name="auc")

model.compile(
    optimizer=opt2,
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[auc_metric]
)

print(model.metrics_names)  # should include 'loss' and 'auc'


['loss', 'compile_metrics']


In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score

y_true, y_pred = [], []
for xb, yb in test_ds:
    y_true.append(yb.numpy())
    y_pred.append(model.predict(xb, verbose=0))

y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)

per_label_auc = []
for j in range(y_true.shape[1]):
    # skip labels with only one class present
    if len(np.unique(y_true[:, j])) < 2:
        per_label_auc.append(np.nan)
    else:
        per_label_auc.append(roc_auc_score(y_true[:, j], y_pred[:, j]))

print("Macro AUROC:", np.nanmean(per_label_auc))
print("Per-label AUROC:", per_label_auc)


Macro AUROC: 0.740584053511194
Per-label AUROC: [np.float64(0.7755781799899448), np.float64(0.8011806751262426), np.float64(0.8246423949917812), np.float64(0.7147966296902467), np.float64(0.735282505702755), np.float64(0.6574560619600834), np.float64(0.6187320103014694), np.float64(0.7970039703270295)]
